In [ ]:
'''
Introdução à Econometria - Uma abordagem moderna (Tradução da 6 edição norte-americana)
Autor: WOOLDRIDGE, J. M.
Editora: CENGAGE LEARNING

Cap. 6: Análise de regressão múltipla: problemas adicionais (Multiple Regression Analysis: Further Issues)
Exemplo 6.7: Previsão de salários de diretores-executivos
             (PREDICTING CEO SALARIES)
             
Arquivo com os dados: ceosal2.xls

Arquivo com dados em:
http://students.cengage.com.br/dashboard/private/livroView.jsf;jsessionid=95E9AD889A4A4B7ABBD2A5251F1E14BE?id=104577

Em caso de dúvidas ou problemas, solicitamos, por gentileza, entrar em contato pelo e-mail:
python.economia@gmail.com
'''

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [2]:
# Em decorrência do objetivo deste exemplo 6.7, iremos importar apenas os dados de salary na forma nível e não na forma logaritmica

df = pd.read_excel('ceosal2.xls',
                   header=None,
                   usecols=[0, 5, 10, 11],
                   names=['salary', 'ceoten', 'lsales', 'lmktval'])

In [3]:
df.head()

,salary,ceoten,lsales,lmktval
0,1161,2,8.732305,10.051910
1,600,10,5.645447,7.003066
2,379,3,5.129899,7.003066
3,651,22,7.003066,6.907755
4,497,6,5.860786,5.958425


### Comandos sugeridos

#### Regressão por Mínimos Quadrados Ordinários (MQO ou OLS na sigla em inglês)

In [4]:
modelo = smf.ols(formula='np.log(salary) ~ lsales + lmktval + ceoten', data=df)
reg = modelo.fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(salary)   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                     26.91
Date:                Sat, 30 Jul 2022   Prob (F-statistic):           2.47e-14
Time:                        09:48:43   Log-Likelihood:                -128.12
No. Observations:                 177   AIC:                             264.2
Df Residuals:                     173   BIC:                             276.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.5038      0.257     17.509      0.000       3.996       5.012
lsales         0.1629      0.039      4.150      0.000       0.085       0.240
lmktval        0.1092      0.050      2.203      0.029       0.011       0.207
ceoten         0.0117      0.005      2.198      0.029       0.001       0.022
==============================================================================
Omnibus:                       25.596   Durbin-Watson:                   2.044
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              123.522
Skew:                          -0.291   Prob(JB):                     1.51e-27
Kurtosis:                       7.051   Cond. No.                         95.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Definição de uma função para previsão de y (nível) quando o modelo tem como variável dependente log(y)

In [ ]:
# Vamos definir uma função para tornar o programa mais simples

In [5]:
def previsao(endog, exog, regressao, tipo):
    '''
    Obtém a previsão de y (na forma nível) quando temos um modelo com a variável dependente log(y)
    endog: Série do pandas
           Série do pandas com os valores observados de y (na forma nível)
    exog: dict
           Dicionário contendo os valores de interesse das variáveis exógenas para obtenção da previsão de y
    regressao: statsmodels.regression.linear_model.RegressionResultsWrapper
           Retorno obtido pela aplicação do método fit()
    tipo: int
          1:'Smearing Estimate'
          2:'Regression Estimate'
    
    returno da funcao: tuple
           (estimativa_alpha_zero, previsao_y_nivel)
    '''
    if tipo == 1:
        log_y = np.log(endog)
        u_chapeu = log_y - regressao.predict()
        exp_u_chapeu = np.exp(u_chapeu)
        soma_exp_u_chapeu = exp_u_chapeu.sum()
        estimativa_alpha_zero =  soma_exp_u_chapeu / regressao.nobs # [Fórmula 6.43]
    if tipo == 2:
        valores_ajustados = regressao.predict()
        m_chapeu_i = np.exp(valores_ajustados)
        modelo_auxiliar = smf.ols(formula=f'{endog.name} ' + '~ - 1 + m_chapeu_i', data=df) # -1 é colocado para retirar o intercepto
        reg_auxiliar = modelo_auxiliar.fit()
        estimativa_alpha_zero = reg_auxiliar.params[0]         # [Fórmula 6.44]
    log_y_chapeu = regressao.predict(exog)
    y_chapeu = estimativa_alpha_zero * np.exp(log_y_chapeu)  # [Fórmula 6.42, utilizando (6.43) ou (6.44)]
    return (estimativa_alpha_zero, y_chapeu[0])


#### Previsão de salary (nível) utilizando o "Duan smearing estimate of alpha zero"

In [6]:
valores_exog = {'lsales': np.log(5000), 'lmktval': np.log(10000), 'ceoten': 10}

previsao(endog=df['salary'], exog=valores_exog, regressao=reg, tipo=1)

(1.135661323998864, 1263.0594799806931)

In [ ]:
# O resultado acima difere do salário previsto no livro (1262,077) em decorrência dos arredondamentos feitos pelo autor do livro

#### Previsão de salary (nível) com o Regression estimate of alpha zero

In [7]:
valores_exog = {'lsales': np.log(5000), 'lmktval': np.log(10000), 'ceoten': 10}

previsao(endog=df['salary'], exog=valores_exog, regressao=reg, tipo=2)

(1.1168566404542437, 1242.1452925225549)

In [ ]:
# O resultado acima difere do salário previsto no livro (1240,968) em decorrência dos arredondamentos feitos pelo autor do livro